In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *
from transformers import set_seed

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_value = 42
set_seed(seed_value)
# supports tasks:
# cb
# rte
# sick
# mrpc
# boolq
# csqa ?????????? or mc?
# argument
# scitail
# imdb
# sst2
# qqp
# mnli


In [3]:
task = "scitail"
model_name = "bert-base-uncased"
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"
#logger = logging.getLogger(__name__)
#logger.setLevel(logging.INFO)

In [4]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

using bert tokenizer
getting encoding:
<function encode_scitail at 0x000002C001F6E550>


In [5]:
#def preprocess_dataset(dataset,encoding_func,tokenizer):
dataset = preprocess_dataset(data,encode,tokenizer)

Map:   0%|          | 0/23097 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not return

mapped
{'train': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask']}


In [6]:
#id2label = {id: label for (id,label) in enumerate(dataset["train"].features["labels"].names)}
#num_labels = len(id2label)
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"


labels 2


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                    
  0%|          | 50/86640 [00:45<8:09:11,  2.95it/s]

{'eval_loss': 0.5120983719825745, 'eval_accuracy': 0.7576687116564417, 'eval_runtime': 26.097, 'eval_samples_per_second': 49.967, 'eval_steps_per_second': 6.246, 'epoch': 0.02}


  0%|          | 100/86640 [01:27<7:49:31,  3.07it/s]

{'eval_loss': 0.482826828956604, 'eval_accuracy': 0.7507668711656442, 'eval_runtime': 26.8769, 'eval_samples_per_second': 48.517, 'eval_steps_per_second': 6.065, 'epoch': 0.03}


                                                       
  0%|          | 150/86640 [02:12<7:49:15,  3.07it/s]

...

                                                       
  1%|          | 650/86640 [09:45<8:40:19,  2.75it/s]

{'eval_loss': 0.3124051094055176, 'eval_accuracy': 0.8734662576687117, 'eval_runtime': 27.8346, 'eval_samples_per_second': 46.848, 'eval_steps_per_second': 5.856, 'epoch': 0.23}


  1%|          | 650/86640 [09:46<21:32:15,  1.11it/s]

{'train_runtime': 586.0818, 'train_samples_per_second': 1182.275, 'train_steps_per_second': 147.829, 'train_loss': 0.4591100810124324, 'epoch': 0.23}


TrainOutput(global_step=650, training_loss=0.4591100810124324, metrics={'train_runtime': 586.0818, 'train_samples_per_second': 1182.275, 'train_steps_per_second': 147.829, 'train_loss': 0.4591100810124324, 'epoch': 0.23})

In [8]:
eval_results = trainer.evaluate()

100%|██████████| 163/163 [00:27<00:00,  5.85it/s]


In [8]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 0.6164497137069702, 'eval_accuracy': 0.6672782874617736, 'eval_runtime': 46.1558, 'eval_samples_per_second': 70.847, 'eval_steps_per_second': 8.861, 'epoch': 0.64}


In [9]:
"""for attr_name, attr_value in vars(default_args).items():
    print(f"{attr_name}: {attr_value}")"""
"""
    train_args = {"label_names":["labels"],
    "evaluation_strategy":"steps",
    "learning_rate":1e-4,
    "num_train_epochs":1,
    "per_device_train_batch_size":8,
    "per_device_eval_batch_size":8,
    "eval_steps":50,
    "logging_steps":200,
    "output_dir":"/eval_results",
    "overwrite_output_dir":True,
    "remove_unused_columns":False,
    "lr_scheduler_type":'linear',
    "load_best_model_at_end":True,
    "metric_for_best_model" : "accuracy",
    "early_stopping_patience":3,
    "save_total_limit":5
}
"""

'\n    train_args = {"label_names":["labels"],\n    "evaluation_strategy":"steps",\n    "learning_rate":1e-4,\n    "num_train_epochs":1,\n    "per_device_train_batch_size":8,\n    "per_device_eval_batch_size":8,\n    "eval_steps":50,\n    "logging_steps":200,\n    "output_dir":"/eval_results",\n    "overwrite_output_dir":True,\n    "remove_unused_columns":False,\n    "lr_scheduler_type":\'linear\',\n    "load_best_model_at_end":True,\n    "metric_for_best_model" : "accuracy",\n    "early_stopping_patience":3,\n    "save_total_limit":5\n}\n'